# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited.

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique
* Implement a K-folds cross validation modeling pipeline for deep learning models
* Apply regularization techniques to improve your model's performance

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab.

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from scikeras.wrappers import KerasRegressor

In this lab you'll be working with the *The Lending Club* data.

- Import the data available in the file `'loan_final.csv'`
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column)
- Print the first five rows of the data
- Print the dimensions of the data

In [2]:
# Import the data
data = pd.read_csv('https://raw.githubusercontent.com/leksea/dsc-tuning-neural-networks-from-start-to-finish-lab-v2-1/master/loan_final.csv')

# Drop rows with no target value

data.dropna(subset=['total_pymnt'], inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [3]:
# Print the dimensions of data
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42.

In [4]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc',
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features)

- Fill all missing values in numeric features with their respective means
- Standardize all the numeric features  
- Convert the final results into DataFrames

In [5]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer(strategy='mean')

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = ss_X.fit_transform(X_train_imputed)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=cont_features)
# Transform test data
X_test_scaled = ss_X.transform(X_test_imputed)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=cont_features)

## Preprocessing (Categorical features)

- Fill all missing values in categorical features with the string `'missing'`
- One-hot encode all categorical features
- Convert the final results into DataFrames


In [6]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat['home_ownership'] = X_train_cat['home_ownership'].fillna('missing')
X_test_cat['home_ownership'] = X_test_cat['home_ownership'].fillna('missing')

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Fit and transform the training data
X_train_ohe = ohe.fit_transform(X_train_cat)

# Transform training and test sets
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names_out(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.toarray(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.toarray(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features.

In [7]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`)

In [8]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps:

- Create a function that returns a compiled deep learning model
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained
- Call the `cross_val_predict()` function to perform k-fold cross-validation

In the cell below, we've defined a baseline model that returns a compiled Keras models.

In [9]:
# Define a function that returns a compiled Keras model
def create_baseline_model():

    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD',
                  loss='mse',
                  metrics=['mse'])

    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and:

- Train for 150 epochs
- Set the batch size to 256

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [10]:
!pip install scikeras

from scikeras.wrappers import KerasRegressor
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with:
- 5-fold cv
- scaled input (`X_train_all`) and output (`y_train_scaled`)

In [23]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9898 - mse: 0.9898
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8594 - mse: 0.8594
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6497 - mse: 0.6497
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5186 - mse: 0.5186
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4513 - mse: 0.4513
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4111 - mse: 0.4111
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3813 - mse: 0.3813
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3450 - mse: 0.3450
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3260 - mse: 0.3260
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3193 - mse: 0.3193
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3058 - mse: 0.3058
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2869 - mse: 0.2869
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0510 - mse: 1.0510
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3459 - mse: 0.3459
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2784 - mse: 0.2784
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2546 - mse: 0.2546
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2381 - mse: 0.2381
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2384 - mse: 0.2384
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2161 - mse: 0.2161
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2280 - mse: 0.2280
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2193 - mse: 0.2193
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2189 - mse: 0.2189
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2171 - mse: 0.2171
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2233 - mse: 0.2233
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8499 - mse: 0.8499
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5599 - mse: 0.5599
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4652 - mse: 0.4652
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4012 - mse: 0.4012
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3795 - mse: 0.3795
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3458 - mse: 0.3458
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3217 - mse: 0.3217
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3001 - mse: 0.3001
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2917 - mse: 0.2917
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2782 - mse: 0.2782
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2614 - mse: 0.2614
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2590 - mse: 0.2590
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7004 - mse: 0.7004
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2574 - mse: 0.2574
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2500 - mse: 0.2500
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2168 - mse: 0.2168
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2334 - mse: 0.2334
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2105 - mse: 0.2105
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2197 - mse: 0.2197
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2250 - mse: 0.2250
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2112 - mse: 0.2112
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2192 - mse: 0.2192
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2178 - mse: 0.2178
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2182 - mse: 0.2182
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8505 - mse: 0.8505
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3375 - mse: 0.3375
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2909 - mse: 0.2909
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2650 - mse: 0.2650
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2474 - mse: 0.2474
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2432 - mse: 0.2432
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2364 - mse: 0.2364
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2309 - mse: 0.2309
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2295 - mse: 0.2295
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2215 - mse: 0.2215
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2302 - mse: 0.2302
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2268 - mse: 0.2268
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

- Find the RMSE on train data

In [24]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.44231738071131554

- Convert the scaled predictions back to original scale
- Calculate RMSE in the original units with `y_train` and `baseline_preds`

In [25]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train, cv_baseline_preds))

15077.840828424274

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [26]:
# Define a function that returns a compiled Keras model
def create_bigger_model():

    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(50, activation='relu'))

    # Third hidden layer
    model.add(layers.Dense(25, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD',
                  loss='mse',
                  metrics=['mse'])

    # Return the compiled model
    return model

In [27]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=150, batch_size=256)

In [28]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5260 - mse: 0.5260
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2171 - mse: 0.2171
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2115 - mse: 0.2115
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2062 - mse: 0.2062
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2061 - mse: 0.2061
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2055 - mse: 0.2055
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2006 - mse: 0.2006
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1970 - mse: 0.1970
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2019 - mse: 0.2019
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1958 - mse: 0.1958
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1972 - mse: 0.1972
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2013 - mse: 0.2013
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5373 - mse: 0.5373
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2337 - mse: 0.2337
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2150 - mse: 0.2150
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2135 - mse: 0.2135
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2124 - mse: 0.2124
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2044 - mse: 0.2044
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2068 - mse: 0.2068
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2057 - mse: 0.2057
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1994 - mse: 0.1994
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2023 - mse: 0.2023
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1987 - mse: 0.1987
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2005 - mse: 0.2005
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4527 - mse: 0.4527
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2108 - mse: 0.2108
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2019 - mse: 0.2019
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2009 - mse: 0.2009
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2008 - mse: 0.2008
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1981 - mse: 0.1981
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1955 - mse: 0.1955
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1961 - mse: 0.1961
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1988 - mse: 0.1988
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2001 - mse: 0.2001
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1939 - mse: 0.1939
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1937 - mse: 0.1937
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5875 - mse: 0.5875
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2327 - mse: 0.2327
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2114 - mse: 0.2114
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2106 - mse: 0.2106
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1990 - mse: 0.1990
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2110 - mse: 0.2110
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2035 - mse: 0.2035
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2020 - mse: 0.2020
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2091 - mse: 0.2091
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2008 - mse: 0.2008
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2010 - mse: 0.2010
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1970 - mse: 0.1970
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4965 - mse: 0.4965
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2239 - mse: 0.2239
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2104 - mse: 0.2104
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2060 - mse: 0.2060
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2074 - mse: 0.2074
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2080 - mse: 0.2080
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2031 - mse: 0.2031
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2065 - mse: 0.2065
Epoch 9/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2025 - mse: 0.2025
Epoch 10/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2001 - mse: 0.2001
Epoch 11/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1959 - mse: 0.1959
Epoch 12/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1975 - mse: 0.1975
Epoch 13/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/st

In [30]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

0.4371535856316529

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

In [31]:
# Define a function that returns a compiled Keras model
def create_regularized_model():

    # Initialize model
    model = models.Sequential()

    # Input layer with dropout
    model.add(layers.Dropout(0.3, input_shape=(n_features,)))

    # First hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Third hidden layer
    model.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD',
                  loss='mse',
                  metrics=['mse'])

    # Return the compiled model
    return model

In [19]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=150, batch_size=256)

In [20]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds  = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled)

In [21]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)!

In [22]:
# ⏰ This cell may take several mintes to run and you may get a comment from tf about optimizing your machine

In [22]:
# Initialize model
model = models.Sequential()

# First hidden layer
model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

# Second hidden layer
model.add(layers.Dense(5, activation='relu'))

# Output layer
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='SGD',
              loss='mse',
              metrics=['mse'])

# Train the model
model.fit(X_train_all,
          y_train_scaled,
          epochs=150,
          batch_size=256,
          verbose=0)

In [ ]:
# Generate predictions on test data
final_preds_scaled = model.predict(X_test_all)

# Convert the predictions back to original scale
final_preds = ss_y.inverse_transform(final_preds_scaled)

# RMSE on test data (original scale)
np.sqrt(mean_squared_error(y_test, final_preds))

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods.